<a href="https://colab.research.google.com/github/ckusmana21/2002_Final_Project/blob/main/2002_Kusmana_Pipeline_Implementation_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the datasets
file_path1 = r"/content/HDPulse_data_export.csv"
hdpulse_data = pd.read_csv(file_path1)

file_path2 = r"/content/Sentencing_Data_Imported_DS2002.csv"
sentencing_data = pd.read_csv(file_path2)

# Check the first few rows to confirm
print(sentencing_data.head())
print(hdpulse_data.head())


<ipython-input-1-517dc70006fe>:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  sentencing_data = pd.read_csv(file_path2)


   Unnamed: 0  USSCIDN   AGE  AGECAT  CIRCDIST  CITIZEN  CITWHERE  DISTRICT  \
0           1  2492676  34.0     4.0        60      3.0      49.0        67   
1           2  2492679  36.0     5.0        89      3.0      49.0        30   
2           3  2492680  50.0     6.0        91      1.0     211.0        31   
3           4  2492681  41.0     6.0        91      3.0      20.0        31   
4           5  2492684  27.0     3.0        91      3.0      20.0        31   

   DOBMON   DOBYR  ...  USARMHI  USKIDHI  VULVCTHI  XMAXSOR  XMINSOR  DSIND  \
0     4.0  1984.0  ...      0.0      0.0       0.0     37.0     30.0      1   
1     2.0  1982.0  ...      0.0      0.0       0.0      6.0      0.0      1   
2    11.0  1967.0  ...      0.0      0.0       0.0     24.0     18.0      1   
3     3.0  1977.0  ...      0.0      0.0       0.0     27.0     21.0      1   
4     1.0  1991.0  ...      0.0      0.0       0.0     24.0     18.0      1   

   DSJANDC  DSPLEA  DSPSR DSSOR  
0        1      

In [2]:
!pip install pandas sqlalchemy


In [3]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('etl_pipeline.db')

# Store the data into SQL tables
sentencing_data.to_sql('SentencingData', conn, if_exists='replace', index=False)
hdpulse_data.to_sql('HDPulseData', conn, if_exists='replace', index=False)

print("Data has been successfully stored in the SQLite database.")


Data has been successfully stored in the SQLite database.


**ABOVE: Saving data to sqlite. Below: filtering/cleaning**

Is the data stored right? (just to check)

In [4]:
# Query the tables to check the data
query_sentencing = pd.read_sql('SELECT * FROM SentencingData LIMIT 5', conn)
query_hdpulse = pd.read_sql('SELECT * FROM HDPulseData LIMIT 5', conn)

print(query_sentencing)
print(query_hdpulse)


   Unnamed: 0  USSCIDN   AGE  AGECAT  CIRCDIST  CITIZEN  CITWHERE  DISTRICT  \
0           1  2492676  34.0     4.0        60      3.0      49.0        67   
1           2  2492679  36.0     5.0        89      3.0      49.0        30   
2           3  2492680  50.0     6.0        91      1.0     211.0        31   
3           4  2492681  41.0     6.0        91      3.0      20.0        31   
4           5  2492684  27.0     3.0        91      3.0      20.0        31   

   DOBMON   DOBYR  ...  USARMHI  USKIDHI  VULVCTHI  XMAXSOR  XMINSOR  DSIND  \
0     4.0  1984.0  ...      0.0      0.0       0.0     37.0     30.0      1   
1     2.0  1982.0  ...      0.0      0.0       0.0      6.0      0.0      1   
2    11.0  1967.0  ...      0.0      0.0       0.0     24.0     18.0      1   
3     3.0  1977.0  ...      0.0      0.0       0.0     27.0     21.0      1   
4     1.0  1991.0  ...      0.0      0.0       0.0     24.0     18.0      1   

   DSJANDC  DSPLEA  DSPSR DSSOR  
0        1      

Merging the data

In [5]:
# Full mapping data for CIRCDIST to STATE_addition
district_to_state = [
    (1, 'Dist of Columbia'), (2, 'Maine'), (3, 'Massachusetts'), (4, 'New Hampshire'),
    (5, 'Puerto Rico'), (6, 'Rhode Island'), (7, 'Connecticut'), (8, 'New York East'),
    (9, 'New York North'), (10, 'New York South'), (11, 'New York West'), (12, 'Vermont'),
    (13, 'Delaware'), (14, 'New Jersey'), (15, 'Penn. East'), (16, 'Penn. Mid'),
    (17, 'Penn. West'), (18, 'Virgin Islands'), (19, 'Maryland'), (20, 'N Carolina East'),
    (21, 'N Carolina Mid'), (22, 'N Carolina West'), (23, 'South Carolina'),
    (24, 'Virginia East'), (25, 'Virginia West'), (26, 'W Virginia North'),
    (27, 'W Virginia South'), (28, 'Louisiana East'), (29, 'Louisiana Middle'),
    (30, 'Louisiana West'), (31, 'Miss. North'), (32, 'Miss. South'), (33, 'Texas East'),
    (34, 'Texas North'), (35, 'Texas South'), (36, 'Texas West'), (37, 'Kentucky East'),
    (38, 'Kentucky West'), (39, 'Michigan East'), (40, 'Michigan West'), (41, 'Ohio North'),
    (42, 'Ohio South'), (43, 'Tennessee East'), (44, 'Tennessee Mid'), (45, 'Tennessee West'),
    (46, 'Illinois Cent'), (47, 'Illinois North'), (48, 'Illinois South'), (49, 'Indiana North'),
    (50, 'Indiana South'), (51, 'Wisconsin East'), (52, 'Wisconsin West'), (53, 'Arkansas East'),
    (54, 'Arkansas West'), (55, 'Iowa North'), (56, 'Iowa South'), (57, 'Minnesota'),
    (58, 'Missouri East'), (59, 'Missouri West'), (60, 'Nebraska'), (61, 'North Dakota'),
    (62, 'South Dakota'), (63, 'Alaska'), (64, 'Arizona'), (65, 'California Cent'),
    (66, 'California East'), (67, 'California North'), (68, 'California South'),
    (69, 'Guam'), (70, 'Hawaii'), (71, 'Idaho'), (72, 'Montana'), (73, 'Nevada'),
    (74, 'N Mariana Island'), (75, 'Oregon'), (76, 'Washington East'), (77, 'Washington West'),
    (78, 'Colorado'), (79, 'Kansas'), (80, 'New Mexico'), (81, 'Oklahoma East'),
    (82, 'Oklahoma North'), (83, 'Oklahoma West'), (84, 'Utah'), (85, 'Wyoming'),
    (86, 'Alabama Mid'), (87, 'Alabama North'), (88, 'Alabama South'), (89, 'Florida Mid'),
    (90, 'Florida North'), (91, 'Florida South'), (92, 'Georgia Mid'), (93, 'Georgia North'),
    (94, 'Georgia South')
]

# Full mapping data for STATE_addition to STATE_final
region_to_state = [
    ('Dist of Columbia', 'District of Columbia'), ('Maine', 'Maine'), ('Massachusetts', 'Massachusetts'),
    ('New Hampshire', 'New Hampshire'), ('Puerto Rico', 'Puerto Rico'), ('Rhode Island', 'Rhode Island'),
    ('Connecticut', 'Connecticut'), ('New York East', 'New York'), ('New York North', 'New York'),
    ('New York South', 'New York'), ('New York West', 'New York'), ('Vermont', 'Vermont'),
    ('Delaware', 'Delaware'), ('New Jersey', 'New Jersey'), ('Penn. East', 'Pennsylvania'),
    ('Penn. Mid', 'Pennsylvania'), ('Penn. West', 'Pennsylvania'), ('Virgin Islands', 'Virgin Islands'),
    ('Maryland', 'Maryland'), ('N Carolina East', 'North Carolina'), ('N Carolina Mid', 'North Carolina'),
    ('N Carolina West', 'North Carolina'), ('South Carolina', 'South Carolina'), ('Virginia East', 'Virginia'),
    ('Virginia West', 'Virginia'), ('W Virginia North', 'West Virginia'), ('W Virginia South', 'West Virginia'),
    ('Louisiana East', 'Louisiana'), ('Louisiana Middle', 'Louisiana'), ('Louisiana West', 'Louisiana'),
    ('Miss. North', 'Mississippi'), ('Miss. South', 'Mississippi'), ('Texas East', 'Texas'),
    ('Texas North', 'Texas'), ('Texas South', 'Texas'), ('Texas West', 'Texas'),
    ('Kentucky East', 'Kentucky'), ('Kentucky West', 'Kentucky'), ('Michigan East', 'Michigan'),
    ('Michigan West', 'Michigan'), ('Ohio North', 'Ohio'), ('Ohio South', 'Ohio'),
    ('Tennessee East', 'Tennessee'), ('Tennessee Mid', 'Tennessee'), ('Tennessee West', 'Tennessee'),
    ('Illinois Cent', 'Illinois'), ('Illinois North', 'Illinois'), ('Illinois South', 'Illinois'),
    ('Indiana North', 'Indiana'), ('Indiana South', 'Indiana'), ('Wisconsin East', 'Wisconsin'),
    ('Wisconsin West', 'Wisconsin'), ('Arkansas East', 'Arkansas'), ('Arkansas West', 'Arkansas'),
    ('Iowa North', 'Iowa'), ('Iowa South', 'Iowa'), ('Minnesota', 'Minnesota'),
    ('Missouri East', 'Missouri'), ('Missouri West', 'Missouri'), ('Nebraska', 'Nebraska'),
    ('North Dakota', 'North Dakota'), ('South Dakota', 'South Dakota'), ('Alaska', 'Alaska'),
    ('Arizona', 'Arizona'), ('California Cent', 'California'), ('California East', 'California'),
    ('California North', 'California'), ('California South', 'California'), ('Guam', 'Guam'),
    ('Hawaii', 'Hawaii'), ('Idaho', 'Idaho'), ('Montana', 'Montana'), ('Nevada', 'Nevada'),
    ('N Mariana Island', 'Northern Mariana Islands'), ('Oregon', 'Oregon'),
    ('Washington East', 'Washington'), ('Washington West', 'Washington'),
    ('Colorado', 'Colorado'), ('Kansas', 'Kansas'), ('New Mexico', 'New Mexico'),
    ('Oklahoma East', 'Oklahoma'), ('Oklahoma North', 'Oklahoma'), ('Oklahoma West', 'Oklahoma'),
    ('Utah', 'Utah'), ('Wyoming', 'Wyoming'), ('Alabama Mid', 'Alabama'), ('Alabama North', 'Alabama'),
    ('Alabama South', 'Alabama'), ('Florida Mid', 'Florida'), ('Florida North', 'Florida'),
    ('Florida South', 'Florida'), ('Georgia Mid', 'Georgia'), ('Georgia North', 'Georgia'),
    ('Georgia South', 'Georgia')
]

# Insert into SQLite
cursor = conn.cursor()

# Create and populate CIRCDISTMapping table
cursor.execute('DROP TABLE IF EXISTS CIRCDISTMapping')
cursor.execute('CREATE TABLE CIRCDISTMapping (CIRCDIST INT, STATE_addition TEXT)')
cursor.executemany('INSERT INTO CIRCDISTMapping VALUES (?, ?)', district_to_state)

# Create and populate RegionToStateMapping table
cursor.execute('DROP TABLE IF EXISTS RegionToStateMapping')
cursor.execute('CREATE TABLE RegionToStateMapping (STATE_addition TEXT, STATE_final TEXT)')
cursor.executemany('INSERT INTO RegionToStateMapping VALUES (?, ?)', region_to_state)

conn.commit()


Run SQL for ETL steps - filter the data based on columns

In [6]:
# Filter the data based on XCRHISSR, OFFGUIDE, and COMBDRG2
cursor.execute('''
    CREATE TABLE FilteredSentencingData AS
    SELECT *
    FROM SentencingData
    WHERE XCRHISSR = 6
      AND OFFGUIDE = 10
      AND COMBDRG2 IN (1, 2)
''')


Mapping CIRCDIST TO STATE_addition

In [7]:
# Map CIRCDIST to STATE_addition
cursor.execute('''
    CREATE TABLE SentencingWithStateAddition AS
    SELECT f.*, c.STATE_addition
    FROM FilteredSentencingData f
    LEFT JOIN CIRCDISTMapping c
    ON f.CIRCDIST = c.CIRCDIST
''')


Map STATE_Addition to STATE_final

In [8]:
# Map STATE_addition to STATE_final
cursor.execute('''
    CREATE TABLE SentencingWithStateFinal AS
    SELECT s.*, r.STATE_final
    FROM SentencingWithStateAddition s
    LEFT JOIN RegionToStateMapping r
    ON s.STATE_addition = r.STATE_addition
''')


Merging with HDPulseData

In [9]:
# Merge with HDPulseData on STATE_final
cursor.execute('''
    CREATE TABLE MergedData AS
    SELECT s.*, h.*
    FROM SentencingWithStateFinal s
    LEFT JOIN HDPulseData h
    ON s.STATE_final = h.State
''')


**Verifying the result**

In [10]:
# Load the merged data into a pandas DataFrame
merged_data = pd.read_sql('SELECT * FROM MergedData', conn)
print(merged_data.head())


   Unnamed: 0  USSCIDN   AGE  AGECAT  CIRCDIST  CITIZEN  CITWHERE  DISTRICT  \
0          84  2492927  39.0     5.0        43      1.0     211.0        49   
1         192  2493242  39.0     5.0        54      1.0     211.0        61   
2         458  2493594  26.0     3.0        52      1.0     211.0        58   
3         507  2493644  35.0     4.0        28      1.0     211.0        35   
4         588  2493871  37.0     5.0        15      1.0     211.0        13   

   DOBMON   DOBYR  ...  DSJANDC  DSPLEA  DSPSR  DSSOR  STATE_addition  \
0     4.0  1979.0  ...        1       1      1      1  Tennessee East   
1     8.0  1979.0  ...        1       1      1      1   Arkansas West   
2    12.0  1991.0  ...        1       1      1      1  Wisconsin West   
3     7.0  1983.0  ...        1       5      1      1  Louisiana East   
4     8.0  1981.0  ...        1       1      1      1      Penn. East   

    STATE_final         State   FIPS  Value (Dollars)  \
0     Tennessee     Tennessee

If the data needs to be saved locally

In [ ]:
from google.colab import files

# Save and download the SQLite file
conn.close()
files.download('etl_pipeline.db')


**TO DO**



*   Use mysql instead of sqlite
*   "Ensure the ETL script can handle updates to data sources and is designed for reproducibility"
* Need to add comment



